# 사이킷런으로 시작하는 머신러닝

## Model Selection module
* 학습데이터와 테스트 데이터세트를 분리하거나 교차검증 분할 및 평가
* 하이퍼파라미터 튜닝을 위한 다양한 함수와 클래스 제공

### 학습/테스트 데이터 세트 분리  - train_test_split()
 - 학습 데이터로만 학습하고 예측하는 경우?

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [17]:
# 데이터셋 로드
iris_data = load_iris()

# 의사결정 트리 사용
dt_clf = DecisionTreeClassifier()
train_data = iris_data.data
train_label = iris_data.target

# 학습을 train_data로 수행
dt_clf.fit(train_data, train_label)

# 학습 데이터 셋으로 예측 수행
pred = dt_clf.predict(train_data)
print("예측 정확도 : ", accuracy_score(train_label, pred))

예측 정확도 :  1.0


**예측정확도가 100%로 나왔다.**

**모의고사를 여러분 치고 그 결과치를 가지고 예측했기 때문에**, 

당연히 예측정확도가 좋게 나온다.

따라서 예측을 수행하는 데이터세트는 학습을 수행한 학습용 데이터세트가 아닌 전용의 테스트 데이터 세트이여야 함.

#### train_test_split()로 학습데이터와 검증 데이터 세트 분리
* 테스트 데이터세트를 전체의 30%, 학습데이터를 70%로 분리

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

In [19]:
dt_clf = DecisionTreeClassifier()
iris_data = load_iris()

# random_state는 호출할때마다 동일한 학습/테스트용 데이터세트를 생성하기 위해서 주어지는 난수값.
# seed()와 동일.

# 피처용 학습데이터/테스트 데이터
# 레이블용 학습/테스트 데이터
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=.2, random_state=2021)

#### 위의 비율을 기반으로 학습 및 예측

In [21]:
# 분리된 학습/테스트용 데이터셋으로 학습
dt_clf.fit(X_train, y_train) 

# 예측 수행
pred = dt_clf.predict(X_test)  # 테스트 데이터 

# 예측 정확도
print("예측 정확도 : ", accuracy_score(y_test,pred))

예측 정확도 :  0.9666666666666667


**넘파이 ndarray뿐만 아니라,판다스 DataFrame/Series도 train_test_split()으로 분할 가능**

In [22]:
import pandas as pd

In [28]:
iris_df = pd.DataFrame(iris_data.data, columns = iris_data.feature_names)
iris_df['target'] = iris_data.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [35]:
ftr_df = iris_df.iloc[:,:-1] # target컬럼은 제외
tgt_df = iris_df.iloc[:, -1]  # target데이터만 

X_train, X_test, y_train, y_test = train_test_split(ftr_df, tgt_df, test_size = .3, random_state = 2021)

In [31]:
display(type(X_train), type(X_test), type(y_train), type(y_test))

numpy.ndarray

pandas.core.frame.DataFrame

pandas.core.series.Series

pandas.core.series.Series

In [37]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)

pred = dt_clf.predict(X_test)
print("예측 정확도  : {0:.4f}".format(accuracy_score(y_test, pred)))

예측 정확도  : 0.9778


### 2.4.2 교차검증

#### 교차검증의 배경
알고리즘을 학습시키는 학습데이터와 이에 대한 예측성능을 평가하기 위한 별도의 테스트 데이터가 필요하지만 이 방법 역시 과적합(ovefitting)에 취약하다. 과적합은 모델이 학습데이터에만 과도하게 최적화되어, 실제 예측을 다른 데이터로 수행할 경우엔 예측성능이 과도하게 떨어지는 것을 말합니다. 
그런데 고정된 학습데이터와 테스트 데이터로 평가하다보면 테스트 데이터에만 과적합되는 학습모델이 만들어져 다른 테스트 데이터가 들어올 경우엔 성능이 저하됩니다. 이런 문제점을 개선하기 위해 교차검증이 나오게 되었다.
교차검증은 본고사를 치르기전에 모의고사를 여러번 쳐본다는 의미로 이해하지.

#### 교차 검증의 종류
- KFold 교차 검증
- StratifiedKFold 검증<br>
  불균형한(imbalanced)분포도를 가진 레이블(결정 클래스) 데이터 집합을 위한 K폴드 방식<br>
  학습 데이터와 검증 데이터 세트가 가지는 레이블 분포도가 유사하도록 검증 데이터 추출<br>
  ex)신용카드 데이터셋에서의 사기 검출(10000건에 신용카드 사용 데이터셋에 사기 건수가 극히 적다)

#### 교차 검증
* K폴드

In [38]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

In [46]:
iris = load_iris()
features = iris.data
label = iris.target

In [45]:
dt_clf = DecisionTreeClassifier(random_state = 2021)

# 5개의 폴드 세트로 분리하는 KFold객체와 폴드 세트별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits = 5)
cv_accuracy = []
print("붓꽃 데이터세트 크기 :", features.shape[0])

붓꽃 데이터세트 크기 : 150


**총 30개씩 5개의 폴드세트가 만들어졌다.**
**한 폴드의 학습용 - 120개, 테스트용은 30개 이렇게 구성된다.**

In [47]:
help(kfold)

Help on KFold in module sklearn.model_selection._split object:

class KFold(_BaseKFold)
 |  KFold(n_splits=5, *, shuffle=False, random_state=None)
 |  
 |  K-Folds cross-validator
 |  
 |  Provides train/test indices to split data in train/test sets. Split
 |  dataset into k consecutive folds (without shuffling by default).
 |  
 |  Each fold is then used once as a validation while the k - 1 remaining
 |  folds form the training set.
 |  
 |  Read more in the :ref:`User Guide <cross_validation>`.
 |  
 |  Parameters
 |  ----------
 |  n_splits : int, default=5
 |      Number of folds. Must be at least 2.
 |  
 |      .. versionchanged:: 0.22
 |          ``n_splits`` default value changed from 3 to 5.
 |  
 |  shuffle : bool, default=False
 |      Whether to shuffle the data before splitting into batches.
 |      Note that the samples within each split will not be shuffled.
 |  
 |  random_state : int or RandomState instance, default=None
 |      When `shuffle` is True, `random_state` a

In [51]:
# KFold객체의 split()를 호출하면서 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환
n_iter = 1
for train_idx, test_idx in kfold.split(features):
    #kfold.split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    train_size = X_train.shape[0]
    test_size  = X_test.shape[0]
    
    # 반복시 마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    print("\n#{0} 교차 검증 정확도 : {1}, 학습데이터 크기 : {2}, 검증 데이터 크기 : {3}".format(n_iter, accuracy, train_size, test_size))
    print("\n#{0} 교차 검증세트 인덱스 : {1}".format(n_iter, test_idx))
    cv_accuracy.append(accuracy)
    n_iter += 1
    
    
# 개별 iteration별 정화도를 합하여 평균 계산
print("\n## 평균 검증 정확도 : ",np.mean(cv_accuracy))


#1 교차 검증 정확도 : 1.0, 학습데이터 크기 : 120, 검증 데이터 크기 : 30

#1 교차 검증세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 : 1.0, 학습데이터 크기 : 120, 검증 데이터 크기 : 30

#2 교차 검증세트 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 : 0.8333, 학습데이터 크기 : 120, 검증 데이터 크기 : 30

#3 교차 검증세트 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 : 0.9333, 학습데이터 크기 : 120, 검증 데이터 크기 : 30

#4 교차 검증세트 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 : 0.8, 학습데이터 크기 : 120, 검증 데이터 크기 : 30

#5 교차 검증세트 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도 :  0.9133266666666667


In [7]:
help(KFold())

Help on KFold in module sklearn.model_selection._split object:

class KFold(_BaseKFold)
 |  KFold(n_splits=5, *, shuffle=False, random_state=None)
 |  
 |  K-Folds cross-validator
 |  
 |  Provides train/test indices to split data in train/test sets. Split
 |  dataset into k consecutive folds (without shuffling by default).
 |  
 |  Each fold is then used once as a validation while the k - 1 remaining
 |  folds form the training set.
 |  
 |  Read more in the :ref:`User Guide <cross_validation>`.
 |  
 |  Parameters
 |  ----------
 |  n_splits : int, default=5
 |      Number of folds. Must be at least 2.
 |  
 |      .. versionchanged:: 0.22
 |          ``n_splits`` default value changed from 3 to 5.
 |  
 |  shuffle : bool, default=False
 |      Whether to shuffle the data before splitting into batches.
 |      Note that the samples within each split will not be shuffled.
 |  
 |  random_state : int or RandomState instance, default=None
 |      When `shuffle` is True, `random_state` a

### KFold API문서

In [8]:
import numpy as np
from sklearn.model_selection import KFold

X = np.array([[1,2], [3,4], [1,2], [3,4]])
y = np.array([1, 2, 3, 4])
kf = KFold(n_splits = 2)

print(type(kf), kf)

for train_idx, test_idx in kf.split(X):
    print("Train : ",  train_idx, "TEST : ", test_idx)
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

<class 'sklearn.model_selection._split.KFold'> KFold(n_splits=2, random_state=None, shuffle=False)
Train :  [2 3] TEST :  [0 1]
Train :  [0 1] TEST :  [2 3]


* Stratified K폴드 검증

###### K 폴드 문제점 확인
* 불균형한 분포도를 가진 레이블(결정 클래스) 데이터집합을 위한 KFold 결정방식
* 원본 데이터의 분포도를 기반으로 학습 데이터와 테스트데이터 세트 결정한다.(특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 것)
* 불균형한 분포의 데이터세트
 1) 대출사기 분별위한 데이터세트
* Stratified KFold는 KFold가 일률적으로 학습데이터와 테스트세트를 분리함으로 인해 필연적으로 나올수밖에 없는 편중된 데이터에 대한 분배문제를 해결하기위해 나온 방식

In [55]:
# 0 - setoa, 1 - Versicolor, 2 - Virginica
iris = load_iris()
iris_df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
display(iris_df.shape[0])

iris_df['label'] = iris.target
iris_df['label'].value_counts()

150

2    50
1    50
0    50
Name: label, dtype: int64

In [56]:
kfold = KFold(n_splits = 5)
n_iter = 1
for train_idx, test_idx in kfold.split(iris_df):
    label_train = iris_df['label'].iloc[train_idx]
    label_test  = iris_df['label'].iloc[test_idx]
    print("\n##교차검증 : {0} ".format(n_iter))
    print("학습 레이블 데이터 분포 : \n", label_train.value_counts())
    print("검증 레이블 데이터 분포 : \n", label_test.value_counts())
    n_iter += 1    


##교차검증 : 1 
학습 레이블 데이터 분포 : 
 2    50
1    50
0    20
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 0    30
Name: label, dtype: int64

##교차검증 : 2 
학습 레이블 데이터 분포 : 
 2    50
1    40
0    30
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 0    20
1    10
Name: label, dtype: int64

##교차검증 : 3 
학습 레이블 데이터 분포 : 
 2    50
0    50
1    20
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 1    30
Name: label, dtype: int64

##교차검증 : 4 
학습 레이블 데이터 분포 : 
 0    50
1    40
2    30
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    20
1    10
Name: label, dtype: int64

##교차검증 : 5 
학습 레이블 데이터 분포 : 
 1    50
0    50
2    20
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    30
Name: label, dtype: int64


* 교차검증 첫번째 세트의 경우는 품종이 0(Setosa)는 전혀 학습을 하지 못하는 결과가 나옴.학습모델은 전혀 Setosa를 예측하지 못하는 결과가 나온다.

* KFold시에는 학습데이터의 분포도와 검증데이터의 분포가 전혀 달라져 버리면 결국은 검증 예측 정확도는 `0%`가 될수밖에 없다.이에 대한 대안으로 Stratified K Fold검증이 있는데, 데이터의 분포도를 보고 학습데이터와 검증데이터를 배분한다.

##### Stratified K Fold를 통한 레이블 데이터 분포도를 고려한 학습데이터와 검증데이터 분할

In [57]:
from sklearn.model_selection import StratifiedKFold

In [58]:
help(StratifiedKFold)

Help on class StratifiedKFold in module sklearn.model_selection._split:

class StratifiedKFold(_BaseKFold)
 |  StratifiedKFold(n_splits=5, *, shuffle=False, random_state=None)
 |  
 |  Stratified K-Folds cross-validator
 |  
 |  Provides train/test indices to split data in train/test sets.
 |  
 |  This cross-validation object is a variation of KFold that returns
 |  stratified folds. The folds are made by preserving the percentage of
 |  samples for each class.
 |  
 |  Read more in the :ref:`User Guide <cross_validation>`.
 |  
 |  Parameters
 |  ----------
 |  n_splits : int, default=5
 |      Number of folds. Must be at least 2.
 |  
 |      .. versionchanged:: 0.22
 |          ``n_splits`` default value changed from 3 to 5.
 |  
 |  shuffle : bool, default=False
 |      Whether to shuffle each class's samples before splitting into batches.
 |      Note that the samples within each split will not be shuffled.
 |  
 |  random_state : int or RandomState instance, default=None
 |     

In [61]:
skf = StratifiedKFold(n_splits = 5)

# 인자가 두개인 것은 하나는 feature data이고 다른 하나는 label데이터 
# 결정(label)데이터를 보고 폴드의 분포도를 쪼갤수 있어야 하니까. 결정데이터가 필요하다.
n_iter = 1
for train_idx, test_idx in skf.split(iris_df, iris_df['label']):
    label_train = iris_df['label'].iloc[train_idx]
    label_test  = iris_df['label'].iloc[test_idx]
    
    print("\n##교차검증 : {0} ".format(n_iter))
    print("학습 레이블 데이터 분포 : \n", label_train.value_counts())
    print("검증 레이블 데이터 분포 : \n", label_test.value_counts())   
    n_iter += 1


##교차검증 : 1 
학습 레이블 데이터 분포 : 
 2    40
1    40
0    40
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    10
1    10
0    10
Name: label, dtype: int64

##교차검증 : 2 
학습 레이블 데이터 분포 : 
 2    40
1    40
0    40
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    10
1    10
0    10
Name: label, dtype: int64

##교차검증 : 3 
학습 레이블 데이터 분포 : 
 2    40
1    40
0    40
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    10
1    10
0    10
Name: label, dtype: int64

##교차검증 : 4 
학습 레이블 데이터 분포 : 
 2    40
1    40
0    40
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    10
1    10
0    10
Name: label, dtype: int64

##교차검증 : 5 
학습 레이블 데이터 분포 : 
 2    40
1    40
0    40
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    10
1    10
0    10
Name: label, dtype: int64


* 보는 바와 같이 학습데이터와 비슷하게 검증데이터도 분포도가 비슷함.

#### Stratified K Fold를 이용한 붓꽃데이터 교차 검증

In [72]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
import numpy as np

In [73]:
iris_data = load_iris()
features = iris_data.data
labels   = iris_data.target

In [76]:
dt_clf = DecisionTreeClassifier(random_state = 2021)
skf = StratifiedKFold(n_splits = 5)
n_iter = 1
cv_accuracy = []
for train_idx, test_idx in skf.split(features, labels):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    train_size = X_train.shape[0]
    test_size  = X_test.shape[0]
    
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    print("\n#{0} 교차 검증 정확도 : {1}, 학습데이터 크기 : {2}, 검증 데이터 크기 : {3}".format(n_iter, accuracy, train_size, test_size))
    print("\n#{0} 교차 검증세트 인덱스 : {1}".format(n_iter, test_idx))
    cv_accuracy.append(accuracy)
    n_iter += 1
    
# 개별 iteration별 정화도를 합하여 평균 계산
print("\n## 교차 검증별 정확도 : ", np.round(cv_accuracy, 4))
print("\n## 평균 검증 정확도 : ",np.mean(cv_accuracy))    


#1 교차 검증 정확도 : 0.9667, 학습데이터 크기 : 120, 검증 데이터 크기 : 30

#1 교차 검증세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  50  51  52  53  54  55  56  57
  58  59 100 101 102 103 104 105 106 107 108 109]

#2 교차 검증 정확도 : 0.9667, 학습데이터 크기 : 120, 검증 데이터 크기 : 30

#2 교차 검증세트 인덱스 : [ 10  11  12  13  14  15  16  17  18  19  60  61  62  63  64  65  66  67
  68  69 110 111 112 113 114 115 116 117 118 119]

#3 교차 검증 정확도 : 0.9, 학습데이터 크기 : 120, 검증 데이터 크기 : 30

#3 교차 검증세트 인덱스 : [ 20  21  22  23  24  25  26  27  28  29  70  71  72  73  74  75  76  77
  78  79 120 121 122 123 124 125 126 127 128 129]

#4 교차 검증 정확도 : 1.0, 학습데이터 크기 : 120, 검증 데이터 크기 : 30

#4 교차 검증세트 인덱스 : [ 30  31  32  33  34  35  36  37  38  39  80  81  82  83  84  85  86  87
  88  89 130 131 132 133 134 135 136 137 138 139]

#5 교차 검증 정확도 : 1.0, 학습데이터 크기 : 120, 검증 데이터 크기 : 30

#5 교차 검증세트 인덱스 : [ 40  41  42  43  44  45  46  47  48  49  90  91  92  93  94  95  96  97
  98  99 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도 :  [0.9667 0.9

* 왜곡된 레이블 데이터세트에서는 반드시 Stratified K 폴드를 이용해 교차검증해야 함.
* 회귀(Regression)에서는 Stratified K 폴드가 지원되지 않습니다.
* 회귀의 결정값은 이산값 형태의 레이블이 아니라 연속된 숫자값이기 때문에 결정값별로 분포를 정하는 의미가 없기 때문.

### `cross_val_score()`
* 교차검증의 순서

 1) 폴드세트를 선정 -> 2) for루프에서 반복으로 학습 및 테스트 데이터의 인덱스 추출 -> 3) 반복적으로 학습과 예측을 수행
 
 -> `cross_val_score()`함수로 1), 2), 3)번의 순서를 한번에 진행한다.

In [78]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np

In [84]:
help(cross_val_score)

Help on function cross_val_score in module sklearn.model_selection._validation:

cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)
    Evaluate a score by cross-validation
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    estimator : estimator object implementing 'fit'
        The object to use to fit the data.
    
    X : array-like of shape (n_samples, n_features)
        The data to fit. Can be for example a list, or an array.
    
    y : array-like of shape (n_samples,) or (n_samples, n_outputs),             default=None
        The target variable to try to predict in the case of
        supervised learning.
    
    groups : array-like of shape (n_samples,), default=None
        Group labels for the samples used while splitting the dataset into
        train/test set. Only used in conjunction with a "Group" :term:`cv`

In [79]:
iris_data = load_iris()

data = iris_data.data
label = iris_data.target
dt_clf = DecisionTreeClassifier(random_state=2021)

In [131]:
#성능지표는 정확도, 교차검증 세트는 5개
scores = cross_val_score(dt_clf, data, label, cv = 5, scoring = 'accuracy')
# 개별 iteration별 정화도를 합하여 평균 계산
print("\n## 교차 검증별 정확도 : ", np.round(scores, 4))
print("\n## 평균 검증 정확도 : ",np.mean(scores))    


## 교차 검증별 정확도 :  [0.9667 0.9667 0.9    1.     1.    ]

## 평균 검증 정확도 :  0.9666666666666668


### `cross_validate()`
* 교차검증의 순서

 1) 폴드세트를 선정 -> 2) for루프에서 반복으로 학습 및 테스트 데이터의 인덱스 추출 -> 3) 반복적으로 학습과 예측을 수행
 
 -> `cross_validate()`함수로 1), 2), 3)번의 순서를 한번에 진행한다.

In [133]:
help(cross_validate)

Help on function cross_validate in module sklearn.model_selection._validation:

cross_validate(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', return_train_score=False, return_estimator=False, error_score=nan)
    Evaluate metric(s) by cross-validation and also record fit/score times.
    
    Read more in the :ref:`User Guide <multimetric_cross_validation>`.
    
    Parameters
    ----------
    estimator : estimator object implementing 'fit'
        The object to use to fit the data.
    
    X : array-like of shape (n_samples, n_features)
        The data to fit. Can be for example a list, or an array.
    
    y : array-like of shape (n_samples,) or (n_samples, n_outputs),             default=None
        The target variable to try to predict in the case of
        supervised learning.
    
    groups : array-like of shape (n_samples,), default=None
        Group labels for the samples used while splitt

In [136]:
cv_results  = cross_validate(dt_clf, data, label,cv=5, scoring='accuracy')
display(type(cv_results))

dict

In [139]:
cv_results.keys()

dict_keys(['fit_time', 'score_time', 'test_score'])

In [140]:
cv_results.items()

dict_items([('fit_time', array([0.00200176, 0.00102019, 0.00100064, 0.00100088, 0.00200152])), ('score_time', array([0.00100017, 0.000983  , 0.        , 0.        , 0.        ])), ('test_score', array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ]))])

### `GridSearchCV`
- 사이킷런은 GridSearchCV를 이용해 Classifier나 Regressor와 같은 알고리즘에 사용되는 하이터 파라미터를 순차적으로 입력하면서 편리하게 최적의 하이퍼 파라미터를 도출 할 수 있는 방안을 제공
- 머신러닝 알고리즘을 튜닝(하이퍼 파라미터 튜닝)

In [113]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

In [114]:
help(GridSearchCV)

Help on class GridSearchCV in module sklearn.model_selection._search:

class GridSearchCV(BaseSearchCV)
 |  GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, iid='deprecated', refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
 |  
 |  Exhaustive search over specified parameter values for an estimator.
 |  
 |  Important members are fit, predict.
 |  
 |  GridSearchCV implements a "fit" and a "score" method.
 |  It also implements "predict", "predict_proba", "decision_function",
 |  "transform" and "inverse_transform" if they are implemented in the
 |  estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated grid-search over a parameter grid.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object.
 |      This is assumed to implement the scikit-learn estimator interface.
 |      Either e

In [119]:
#데이터를 로딩하고 학습데이터와 테스트 데이터 분리
iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,test_size = 0.2,random_state = 121)

dtree = DecisionTreeClassifier()

# 파라미터 형태를 딕셔너리 형태로 설정
# 반드시 키에 대한 값의 파라미터 형식은 list타입이어야 함.
grid_parameters = {
                  'max_depth': [1, 2, 3],
                  'min_samples_split': [2, 3]
                  }

In [122]:
import pandas as pd

#grid_parameters의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트 수행 설정
## refit = True가 default임, True이면 가장 좋은 파라미터 설정으로 재학습
grid_dtree = GridSearchCV(dtree,param_grid = grid_parameters, cv = 3, refit = True)
grid_dtree.fit(X_train, y_train)

# GridSearchCV결과를 추출해 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)

scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


* GridSearchCV 최적 파라미터

In [126]:
# 가장 좋은 GridSearchCV의 최적 파라미터 
print("GridSearchCV최적 파라미터 : ", grid_dtree.best_params_)

GridSearchCV최적 파라미터 :  {'max_depth': 3, 'min_samples_split': 2}


* GridSearchCV 최고 정확도

In [127]:
print("GridSearchCV최고 정확도 : ", grid_dtree.best_score_)

GridSearchCV최고 정확도 :  0.975


* 테스트 데이터세트 정확도

In [129]:
# refit=True로 설정된 GridSearchCV객체가 fit()를 수행시 학습이 완료된 Estimator를 내포하고 있으므로 preidct()를 통해 예측도 가능
pred = grid_dtree.predict(X_test)
print("테스트 데이터 세트 정확도 : {0:.4f}".format(accuracy_score(y_test, pred)))

테스트 데이터 세트 정확도 : 0.9667


* GridSearchCV의 Refit으로 이미 학습된 estimator반환

In [130]:
estimator = grid_dtree.best_estimator_
pred = estimator.predict(X_test)
print("테스트 데이터 세트 정확도 : {0:.4f}".format(accuracy_score(y_test, pred)))

테스트 데이터 세트 정확도 : 0.9667


### 정리
* 일반적으로 학습 데이터를 GridSearchCV를 이용해 최적 하이퍼 파라미터 튜닝을 수행후 별도의 테스트에서 이를 평가하는 것이 일반적인 머신러닝 적용방법임